<a href="https://colab.research.google.com/github/CarlOsito16/Hexamind/blob/main/Summarization/Hexamind_resume.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall transformers

Found existing installation: transformers 4.26.0
Uninstalling transformers-4.26.0:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.8/dist-packages/transformers-4.26.0.dist-info/*
    /usr/local/lib/python3.8/dist-packages/transformers/*
Proceed (Y/n)? Y
  Successfully uninstalled transformers-4.26.0


# Install the dependent packages

In [1]:
!pip install transformers[sentencepiece]
!pip install gradio
!pip install sentence_transformers
!python -m spacy download fr_core_news_md

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 77.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Text Summarization

In [5]:
re.__version__

NameError: ignored

In [6]:
!python -m re info

## Declare model tokenizers: `camembert` and `T5`

In [2]:
import torch
from transformers import RobertaTokenizerFast, EncoderDecoderModel, AutoTokenizer, AutoModelForSeq2SeqLM
import re
import gradio as gr

#set the device agnostics code
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# whitespace handler to be used in `t5 model`
WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

article_text = """\"Un nuage de fumée juste après l’explosion, le 1er juin 2019. Une déflagration dans une importante usine d’explosifs du centre de la Russie a fait au moins 79 blessés samedi 1er juin. L’explosion a eu lieu dans l’usine Kristall à Dzerzhinsk, une ville située à environ 400 kilomètres à l’est de Moscou, dans la région de Nijni-Novgorod. « Il y a eu une explosion technique dans l’un des ateliers, suivie d’un incendie qui s’est propagé sur une centaine de mètres carrés », a expliqué un porte-parole des services d’urgence. Des images circulant sur les réseaux sociaux montraient un énorme nuage de fumée après l’explosion. Cinq bâtiments de l’usine et près de 180 bâtiments résidentiels ont été endommagés par l’explosion, selon les autorités municipales. Une enquête pour de potentielles violations des normes de sécurité a été ouverte. Fragments de shrapnel Les blessés ont été soignés après avoir été atteints par des fragments issus de l’explosion, a précisé une porte-parole des autorités sanitaires citée par Interfax. « Nous parlons de blessures par shrapnel d’une gravité moyenne et modérée », a-t-elle précisé. Selon des représentants de Kristall, cinq personnes travaillaient dans la zone où s’est produite l’explosion. Elles ont pu être évacuées en sécurité. Les pompiers locaux ont rapporté n’avoir aucune information sur des personnes qui se trouveraient encore dans l’usine."""


cmb_ckpt = 'mrm8488/camembert2camembert_shared-finetuned-french-summarization'
cmb_tokenizer = RobertaTokenizerFast.from_pretrained(cmb_ckpt)
cmb_model = EncoderDecoderModel.from_pretrained(cmb_ckpt).to(device)


t5_model_name = "csebuetnlp/mT5_multilingual_XLSum"
t5_tokenizer = AutoTokenizer.from_pretrained(t5_model_name)
t5_model = AutoModelForSeq2SeqLM.from_pretrained(t5_model_name)




def clear_input():
    return ("", "")

    
# def camembert_generate_summary(article_text):
#    inputs = cmb_tokenizer([article_text], padding="max_length", truncation=True,
#                           max_length=512,
#                           return_tensors="pt")
#    input_ids = inputs.input_ids.to(device)
#    attention_mask = inputs.attention_mask.to(device)
#    output = cmb_model.generate(input_ids, attention_mask=attention_mask)
#    return cmb_tokenizer.decode(output[0], skip_special_tokens=True)

       
def camembert_generate_summary(article_text):
   inputs = cmb_tokenizer([article_text], padding=250, truncation=True,
                          max_length=250,
                          return_tensors="pt")
   input_ids = inputs.input_ids.to(device)
   attention_mask = inputs.attention_mask.to(device)
   output = cmb_model.generate(input_ids, attention_mask=attention_mask)
   return cmb_tokenizer.decode(output[0], skip_special_tokens=True)


def t5_generate_summary(article_text):
    input_ids = t5_tokenizer(
    [WHITESPACE_HANDLER(article_text)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512)["input_ids"]
    
    output_ids = t5_model.generate(
        input_ids=input_ids,
        max_length=84,
        no_repeat_ngram_size=2,
        num_beams=4
    )[0]
    
    output = t5_tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
    )

    return output

def summarizer(dropdown_model, article_text):
    """
    Ruturs a summarized version from the full article based on the selected pretrained-model
    """

    if dropdown_model == 'camembert':
        summary = camembert_generate_summary(article_text)

    elif dropdown_model == 'T5':
        summary = t5_generate_summary(article_text)

    return summary



The following encoder weights were not tied to the decoder ['roberta/pooler']
The following encoder weights were not tied to the decoder ['roberta/pooler']


/usr/local/lib/python3.8/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [22]:
       
def camembert_generate_summary(article_text):
   inputs = cmb_tokenizer([article_text], padding="max_length", truncation=True,
                          max_length=250,
                          return_tensors="pt")
   input_ids = inputs.input_ids.to(device)
   attention_mask = inputs.attention_mask.to(device)
   output = cmb_model.generate(input_ids, attention_mask=attention_mask)
   return cmb_tokenizer.decode(output[0], skip_special_tokens=True)

In [10]:
t5_model_name = "csebuetnlp/mT5_multilingual_XLSum"
t5_tokenizer = AutoTokenizer.from_pretrained(t5_model_name)
t5_model = AutoModelForSeq2SeqLM.from_pretrained(t5_model_name)

In [3]:
camembert_generate_summary("hello, its me")

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 64 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


"Notre sélection de photographies d'actualité des sept derniers jours à travers le monde."

In [14]:
t5_generate_summary(article_text)

'Les habitants de Dijon, dans le centre de la France, sont tombés indemnes à des dégâts.'

In [24]:
article_text = """
Tv commandé le 23 novembre pendant le black friday avec une livraison un peu tardive le 2 decembre, tv qui arrive cassée donc refus au pres du livreur et à partir de la les problemes commencent, appel tous les jours sur leur plateforme, aucune possibilité d'avoir unresponsable donc que des conseillers aussi mauvais l'un que l'autre qui pour calmer le jeu me racontent que la tv est bien parti et que le livreur devrait m'appeler rapidement pour organiser un rdv. Aucun suivi du dossier alors que je ne demandait qu'un renvoi et pas de remboursement le probleme est resté jusqu'au 18 decembre pour avoir une reponse par mail avec un remboursement soit presque 1 mois ou l'on m'a fait tourner en rond.Sav plus que mauvais, 1ere et derniere expérience chez eux....


"""

In [25]:
summarizer('camembert', article_text)

"La réception du colis, qui devait être expédié le 23 novembre, n'a pas pu être l'ordre voulu."

## 🎬Demo: on text summarization

In [3]:
import gradio as gr

In [4]:
with gr.Blocks() as demo:
    gr.Markdown("Summarize the article text.")
    with gr.Tab("Each model"):
        with gr.Row():
            with gr.Column():
                input_models = gr.Dropdown(['camembert', 'T5'])
                input_article = gr.TextArea(label = 'Article to be summarized')
            with gr.Column():
                summarized_output = gr.TextArea(label= 'Generated summa')
        with gr.Row():

            clear_button = gr.Button("Clear")
            summarize_button = gr.Button("Summarize!")
            
    summarize_button.click(summarizer,
                           inputs = [input_models, input_article] ,
                           outputs = summarized_output)
    
    clear_button.click(clear_input,
                       outputs = [input_models, input_article])
    
    example = "Un nuage de fumée juste après l’explosion, le 1er juin 2019. Une déflagration dans une importante usine d’explosifs du centre de la Russie a fait au moins 79 blessés samedi 1er juin. L’explosion a eu lieu dans l’usine Kristall à Dzerzhinsk, une ville située à environ 400 kilomètres à l’est de Moscou, dans la région de Nijni-Novgorod. « Il y a eu une explosion technique dans l’un des ateliers, suivie d’un incendie qui s’est propagé sur une centaine de mètres carrés », a expliqué un porte-parole des services d’urgence. Des images circulant sur les réseaux sociaux montraient un énorme nuage de fumée après l’explosion. Cinq bâtiments de l’usine et près de 180 bâtiments résidentiels ont été endommagés par l’explosion, selon les autorités municipales. Une enquête pour de potentielles violations des normes de sécurité a été ouverte. Fragments de shrapnel Les blessés ont été soignés après avoir été atteints par des fragments issus de l’explosion, a précisé une porte-parole des autorités sanitaires citée par Interfax. « Nous parlons de blessures par shrapnel d’une gravité moyenne et modérée », a-t-elle précisé. Selon des représentants de Kristall, cinq personnes travaillaient dans la zone où s’est produite l’explosion. Elles ont pu être évacuées en sécurité. Les pompiers locaux ont rapporté n’avoir aucune information sur des personnes qui se trouveraient encore dans l’usine."
    examples = gr.Examples(examples=[ ["camembert",example],
                                     ["T5",example]],
                                     inputs=[input_models, input_article])

demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://58eff0b2-3cdc-4e6d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://58eff0b2-3cdc-4e6d.gradio.live


# Keyword Extraction

## Call the CountVectorizer

In [5]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
article_text = "Je peux pas mettre zéro mais sinon j'aurai mis cette note. Depuis décembre j'attends le remboursement d'un ordinateur portable qui n'a jamais été livré. A la place une cafetière. Une semaine pour avoir un bon de retour. Le colis a bien été remis à l'entrepôt et après 10 appels des échanges de mail aujourd'hui on vient de me dire que je ne serai pas remboursée. Je comprends pas la blague. On ne me rembourse pas pour des erreurs d'argent de chez Carrefour. Je ne peux pas travailler de la maison car je n'ai pas d'ordinateur et je peux pas en racheter un car j'ai pas l'argent enfin di il est sur les comptes de carrefour. Je peux porter plainte? Numero de commande 607275144"
n_gram_range = (1, 1)

# Extract candidate words/phrases
count = CountVectorizer(ngram_range=n_gram_range).fit([article_text])
candidates = count.get_feature_names_out()
candidates

array(['10', '607275144', 'ai', 'appels', 'après', 'argent', 'attends',
       'aujourd', 'aurai', 'avoir', 'bien', 'blague', 'bon', 'cafetière',
       'car', 'carrefour', 'cette', 'chez', 'colis', 'commande',
       'comprends', 'comptes', 'de', 'depuis', 'des', 'di', 'dire',
       'décembre', 'en', 'enfin', 'entrepôt', 'erreurs', 'est', 'et',
       'hui', 'il', 'jamais', 'je', 'la', 'le', 'les', 'livré', 'mail',
       'mais', 'maison', 'me', 'mettre', 'mis', 'ne', 'note', 'numero',
       'on', 'ordinateur', 'pas', 'peux', 'place', 'plainte', 'portable',
       'porter', 'pour', 'que', 'qui', 'racheter', 'rembourse',
       'remboursement', 'remboursée', 'remis', 'retour', 'semaine',
       'serai', 'sinon', 'sur', 'travailler', 'un', 'une', 'vient',
       'zéro', 'échanges', 'été'], dtype=object)

In [7]:
article_text = """
Attention à contrôler vos surgelés à réception d’une commande en ligne, livrée chez vous.
Congélateur plein, j’ai ouvert les boites pour gagner de la place et mauvaise surprise : tout était décongelé !!
"""
n_gram_range = (1, 1)

# Extract candidate words/phrases
count = CountVectorizer(ngram_range=n_gram_range).fit([article_text])
candidates = count.get_feature_names_out()
candidates

array(['ai', 'attention', 'boites', 'chez', 'commande', 'congélateur',
       'contrôler', 'de', 'décongelé', 'en', 'et', 'gagner', 'la', 'les',
       'ligne', 'livrée', 'mauvaise', 'ouvert', 'place', 'plein', 'pour',
       'réception', 'surgelés', 'surprise', 'tout', 'une', 'vos', 'vous',
       'était'], dtype=object)

In [8]:
print(candidates)
print(len(candidates))


['ai' 'attention' 'boites' 'chez' 'commande' 'congélateur' 'contrôler'
 'de' 'décongelé' 'en' 'et' 'gagner' 'la' 'les' 'ligne' 'livrée'
 'mauvaise' 'ouvert' 'place' 'plein' 'pour' 'réception' 'surgelés'
 'surprise' 'tout' 'une' 'vos' 'vous' 'était']
29


## Request to Github to download `french_stopwords.txt`

In [9]:
import requests
from pathlib import Path

if Path('french_stopword.txt').is_file():
    print('already exists')
else:
    print('not existed yet')
    request = requests.get("https://raw.githubusercontent.com/stopwords-iso/stopwords-fr/master/stopwords-fr.txt")
    with open('french_stopword.txt', "wb") as f:
        f.write(request.content)

not existed yet


In [10]:
# opening the file in read mode
my_file = open("french_stopword.txt", "r")
  
# reading the file
data = my_file.read()
  
# replacing end of line('/n') with ' ' and
# splitting the text it further when '.' is seen.
french_stopwords_list = data.replace('\n', ' ').split(" ")
  
# # printing the data
print(french_stopwords_list)
my_file.close()

['a', 'abord', 'absolument', 'afin', 'ah', 'ai', 'aie', 'aient', 'aies', 'ailleurs', 'ainsi', 'ait', 'allaient', 'allo', 'allons', 'allô', 'alors', 'anterieur', 'anterieure', 'anterieures', 'apres', 'après', 'as', 'assez', 'attendu', 'au', 'aucun', 'aucune', 'aucuns', 'aujourd', "aujourd'hui", 'aupres', 'auquel', 'aura', 'aurai', 'auraient', 'aurais', 'aurait', 'auras', 'aurez', 'auriez', 'aurions', 'aurons', 'auront', 'aussi', 'autant', 'autre', 'autrefois', 'autrement', 'autres', 'autrui', 'aux', 'auxquelles', 'auxquels', 'avaient', 'avais', 'avait', 'avant', 'avec', 'avez', 'aviez', 'avions', 'avoir', 'avons', 'ayant', 'ayez', 'ayons', 'b', 'bah', 'bas', 'basee', 'bat', 'beau', 'beaucoup', 'bien', 'bigre', 'bon', 'boum', 'bravo', 'brrr', 'c', 'car', 'ce', 'ceci', 'cela', 'celle', 'celle-ci', 'celle-là', 'celles', 'celles-ci', 'celles-là', 'celui', 'celui-ci', 'celui-là', 'celà', 'cent', 'cependant', 'certain', 'certaine', 'certaines', 'certains', 'certes', 'ces', 'cet', 'cette', 'ce

In [11]:
article_text = "Je peux pas mettre zéro mais sinon j'aurai mis cette note. Depuis décembre j'attends le remboursement d'un ordinateur portable qui n'a jamais été livré. A la place une cafetière. Une semaine pour avoir un bon de retour. Le colis a bien été remis à l'entrepôt et après 10 appels des échanges de mail aujourd'hui on vient de me dire que je ne serai pas remboursée. Je comprends pas la blague. On ne me rembourse pas pour des erreurs d'argent de chez Carrefour. Je ne peux pas travailler de la maison car je n'ai pas d'ordinateur et je peux pas en racheter un car j'ai pas l'argent enfin di il est sur les comptes de carrefour. Je peux porter plainte? Numero de commande 607275144"
n_gram_range = (1, 1)

# Extract candidate words/phrases
count = CountVectorizer(ngram_range=n_gram_range,
                        stop_words = french_stopwords_list).fit([article_text])
candidates_without_stopwords = count.get_feature_names_out()

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['quelqu'] not in stop_words.
  warnings.warn(


In [12]:
len(candidates_without_stopwords)

39

## prin the the `candidates_without_stopwords`

In [13]:
candidates_without_stopwords

array(['10', '607275144', 'appels', 'argent', 'attends', 'blague',
       'cafetière', 'carrefour', 'colis', 'commande', 'comprends',
       'comptes', 'di', 'décembre', 'entrepôt', 'erreurs', 'jamais',
       'livré', 'mail', 'maison', 'mettre', 'mis', 'note', 'numero',
       'ordinateur', 'place', 'plainte', 'portable', 'porter', 'racheter',
       'rembourse', 'remboursement', 'remboursée', 'remis', 'semaine',
       'travailler', 'vient', 'zéro', 'échanges'], dtype=object)

# `SpaCy` for name entity recognition

In [14]:
import spacy

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [15]:
nlp = spacy.load("fr_core_news_md")

In [16]:
for word_idx in range(10):
    doc = nlp(candidates_without_stopwords[word_idx])
    print(doc.text)
    for token in doc:
        print(token.text, token.pos_, token.dep_)

10
10 NUM ROOT
607275144
607275144 DET ROOT
appels
appels NOUN ROOT
argent
argent NOUN ROOT
attends
attends VERB ROOT
blague
blague NOUN ROOT
cafetière
cafetière NOUN ROOT
carrefour
carrefour NOUN ROOT
colis
colis NOUN ROOT
commande
commande NOUN ROOT


## Functionalize the Noun only

In [17]:
def slice_only_noun_token(ner, token_list):
    slice_list = []
    for word_idx in range(len(token_list)):
         doc = nlp(candidates_without_stopwords[word_idx])

         for token in doc:
             if token.pos_ == 'NOUN':
                 slice_list.append(token.text)

    return slice_list


In [18]:
sliced_candidates_without_stopwords = slice_only_noun_token(nlp, candidates_without_stopwords)
sliced_candidates_without_stopwords

['appels',
 'argent',
 'blague',
 'cafetière',
 'carrefour',
 'colis',
 'commande',
 'comprends',
 'comptes',
 'décembre',
 'entrepôt',
 'erreurs',
 'mail',
 'maison',
 'note',
 'numero',
 'ordinateur',
 'place',
 'plainte',
 'portable',
 'remboursement',
 'semaine',
 'échanges']

# Top 3 Keyword

In [19]:
from sentence_transformers import SentenceTransformer
model =  SentenceTransformer("dangvantuan/sentence-camembert-large")

In [20]:
from sklearn.metrics.pairwise import cosine_similarity


doc_embedding = model.encode([article_text])
candidate_embeddings = model.encode(sliced_candidates_without_stopwords)

top_n = 3
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [sliced_candidates_without_stopwords[index] for index in distances.argsort()[0][-top_n:]]


In [21]:
keywords

['ordinateur', 'colis', 'remboursement']

In [22]:
doc_embedding = model.encode([article_text])
candidate_embeddings = model.encode(candidates_without_stopwords)

top_n = 10
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates_without_stopwords[index] for index in distances.argsort()[0][-top_n:]]
keywords

['plainte',
 'jamais',
 'cafetière',
 'ordinateur',
 'colis',
 'livré',
 'attends',
 'remboursée',
 'remboursement',
 'rembourse']

# Functionalize the `key work extraction`

In [23]:
def key_word_extractor(article_text, 
                       ner,
                       french_stopwords=None):
    """
    Extract only the most re
        Args:
            ner (spacy): The NER class to detect the `token.pos_`
            token_list (list): List of token from the full article

        Returns:
            slice_list (list): List of token containing only "NOUN" part of speech
    """



    n_gram_range = (1, 1) #specify the key word length
    count = CountVectorizer(ngram_range=n_gram_range,
                        stop_words = french_stopwords).fit([article_text])  #call the `CountVectorizer`
    candidates_without_stopwords = count.get_feature_names_out()
    sliced_candidates_without_stopwords = slice_only_noun_token(ner, candidates_without_stopwords )

    return sliced_candidates_without_stopwords


def slice_only_noun_token(ner, token_list):
    """
    Given the tokenized list, this function returns only the "NOUN" token
        Args:
            ner (spacy): The NER class to detect the `token.pos_`
            token_list (list): List of token from the full article

        Returns:
            slice_list (list): List of token containing only "NOUN" part of speech
    """

    slice_list = []
    for word_idx in range(len(token_list)):
         doc = ner(candidates_without_stopwords[word_idx])

         for token in doc:
             if token.pos_ == 'NOUN':
                 slice_list.append(token.text)

    return slice_list

# Creat the `keyWordExtractor class`

In [ ]:
def count_vectorizer(n_gram):
    n_gram_range = (n_gram, n_gram)
        # Extract candidate words/phrases
    count = CountVectorizer(ngram_range=n_gram_range,
                    stop_words = self.french_stopwords).fit([article_text])
    candidates = count.get_feature_names_out()

    return candidates


In [54]:
class keyWordExtractor():
    
    def __init__(self, 
                 article_text,
                 similarity_model,
                 n_gram = 1,
                 top_n = 3,
                 french_stopwords = None,
                 ner= None,
                 ):
        self.article_text = article_text
        self.french_stopwords = french_stopwords
        self.candidates = self.count_vectorizer(n_gram)
        self.noun_candidates, self.proper_noun_candidates = self.slice_only_noun_token(ner, self.candidates)
        self.top_n_keywords = self.top_n_extractor(similarity_model, top_n)
    
    def count_vectorizer(self, n_gram):
        n_gram_range = (n_gram, n_gram)
        # Extract candidate words/phrases
        count = CountVectorizer(ngram_range=n_gram_range,
                        stop_words = self.french_stopwords).fit([self.article_text]) #Main change
        candidates = count.get_feature_names_out()

        return candidates

    def slice_only_noun_token(self, ner, token_list):
        """
        Given the tokenized list, this function returns only the "NOUN" token
            Args:
                ner (spacy): The NER class to detect the `token.pos_`
                token_list (list): List of token from the full article

            Returns:
                slice_list (list): List of token containing only "NOUN" part of speech
        """

        noun_slice_list = []
        proper_noun_slice_list = []
        for word_idx in range(len(token_list)):
            doc = ner(token_list[word_idx])

            for token in doc:
                if token.pos_ == 'NOUN':
                    noun_slice_list.append(token.text)
                elif token.pos_ == 'PROPN':
                    proper_noun_slice_list.append(token.text)

        return noun_slice_list, proper_noun_slice_list

    def top_n_extractor(self, model, top_n):
        doc_embedding = model.encode([self.article_text])
        candidate_embeddings = model.encode(self.noun_candidates)
        distances = cosine_similarity(doc_embedding, candidate_embeddings)
        keywords = [self.noun_candidates[index] for index in distances.argsort()[0][-top_n:]]

        return keywords





In [25]:
article_text= """
 Bonjour,
Je vais vous expliquer mon ressenti concernant mon expérience pour un achat sur le site carrefour.fr.
J'étais très content lors de mon achat
"PC portable IdeaPad 3 15IIL05 LENOVO", tout s'est bien passé en ligne, l'achat correspondait à mes attentes...peu de temps après il n'y en avait plus de disponible (Ce qui est logique quand il n'y en a plus en stock). Mais ce n'était pas mon cas car ma commande était validée et réglée.

"""

a= keyWordExtractor(article_text,
                     n_gram = 1, 
                     top_n = 3,
                     ner = nlp,
                     similarity_model = model)
print(a)

In [26]:
a.proper_noun_candidates

['lenovo', 'pc']

## 🎬 DEMO: Key word Extraction

In [70]:
def extract_top_3(article):
    nlp = spacy.load("fr_core_news_md")
    # model = SentenceTransformer("dangvantuan/sentence-camembert-large") #

    a= keyWordExtractor(article,
                        n_gram = 1, 
                        top_n = 3,
                        ner = nlp,
                        similarity_model = model)
    keyword = ", ".join(a.top_n_keywords) #to return ['a' , 'b'] >> "a, b"
    proper_nonuns = ", ".join(a.proper_noun_candidates)

    return keyword, proper_nonuns

In [73]:
article_text = """

Attention à contrôler vos surgelés à réception d’une commande en ligne, livrée chez vous.
Congélateur plein, j’ai ouvert les boites pour gagner de la place et mauvaise surprise : tout était décongelé !!

"""

print(extract_top_3(article_text))

('surprise, boites, congélateur', '')


In [57]:
article_text = """

je reçois ce matin un sms de carrefour me disant que les soldes sont commencés avec la photo et le prix de plusieurs articles
je suis intéressé par un ordinateur et me présente au magasin carrefour de châteauroux
je montre le message au vendeur qui me dit ne pas être au courant de la promotion de cet article et que c'est une promotion nationale et que le magasin de châteauroux n'en a pas

"""

print(extract_top_3(article_text))

['ordinateur', 'magasin', 'châteauroux']


In [46]:
article_text = """
Bundle Volant+Pédalier logitech, jeu absent dans le bundle, mon fils n'a même pas pu profiter de son cadeau de Noël en plus !!!! ils me demandent le 28/12/2022 de contacter un 01 de SAV, j'ai appelé récemment car entre deux j'ai eu le covid qui a duré. Ce SAV me renvoi vers les Service Client Carrefour car ce n'est pas chez eux. Je recontacte par mail et idem, la même personne que précedemment fait semblant de ne rien comprendre, me dit de rappeler le 01 et ensuite me répond que le délai est dépassé !!!!!! N° de la réclamation 221228-001788.
"""

# print(extract_top_3(article_text).top_n_keywords)
# print(extract_top_3(article_text).proper_noun_candidates)
print(extract_top_3(article_text))

['sav', 'pédalier', 'délai']


In [58]:
article_text = """

"""

# print(extract_top_3(article_text).top_n_keywords)
# print(extract_top_3(article_text).proper_noun_candidates)
print(extract_top_3(article_text))

ValueError: ignored

In [44]:
hello = gr.TextArea(label = 'Article to be summarized')

itsme = gr.Textbox(label="Top 3 keywords")
# output_2 = gr.Textbox(label="Proper Nouns")

demo = gr.Interface(
    fn=extract_top_3,
    inputs= hello,
    outputs= itsme )

demo.launch(debug= True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.


In [41]:
hello = gr.TextArea(label = 'Article to be summarized')

itsme = gr.Textbox(label="Top 3 keywords")
# output_2 = gr.Textbox(label="Proper Nouns")

demo = gr.Interface(
    fn=extract_top_3,
    inputs= hello,
    outputs= itsme )

demo.launch(debug= True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.


---

# COMBINE 2 tabs of demos together

In [74]:
with gr.Blocks() as demo:
    gr.Markdown("Summarize the article text.")
    with gr.Tab("Summarization"):
        with gr.Row():
            with gr.Column():
                input_models = gr.Dropdown(['camembert', 'T5'])
                input_article = gr.TextArea(label = 'Article to be summarized')
            with gr.Column():
                summarized_output = gr.TextArea(label= 'Generated summa')
        with gr.Row():

            clear_button = gr.Button("Clear")
            summarize_button = gr.Button("Summarize!")
            
        summarize_button.click(summarizer,
                            inputs = [input_models, input_article] ,
                            outputs = summarized_output)
        
        clear_button.click(clear_input,
                        outputs = [input_models, input_article])
    
        example = "Un nuage de fumée juste après l’explosion, le 1er juin 2019. Une déflagration dans une importante usine d’explosifs du centre de la Russie a fait au moins 79 blessés samedi 1er juin. L’explosion a eu lieu dans l’usine Kristall à Dzerzhinsk, une ville située à environ 400 kilomètres à l’est de Moscou, dans la région de Nijni-Novgorod. « Il y a eu une explosion technique dans l’un des ateliers, suivie d’un incendie qui s’est propagé sur une centaine de mètres carrés », a expliqué un porte-parole des services d’urgence. Des images circulant sur les réseaux sociaux montraient un énorme nuage de fumée après l’explosion. Cinq bâtiments de l’usine et près de 180 bâtiments résidentiels ont été endommagés par l’explosion, selon les autorités municipales. Une enquête pour de potentielles violations des normes de sécurité a été ouverte. Fragments de shrapnel Les blessés ont été soignés après avoir été atteints par des fragments issus de l’explosion, a précisé une porte-parole des autorités sanitaires citée par Interfax. « Nous parlons de blessures par shrapnel d’une gravité moyenne et modérée », a-t-elle précisé. Selon des représentants de Kristall, cinq personnes travaillaient dans la zone où s’est produite l’explosion. Elles ont pu être évacuées en sécurité. Les pompiers locaux ont rapporté n’avoir aucune information sur des personnes qui se trouveraient encore dans l’usine."
        examples = gr.Examples(examples=[ ["camembert",example],
                                        ["T5",example]],
                                        inputs=[input_models, input_article])
        
    with gr.Tab("Key word extraction"):
        

        with gr.Row():
            with gr.Column():
                input_article = gr.TextArea(label = 'Article to be extracted')
            with gr.Column():
                output_1 = gr.Textbox(label="Top 3 keywords")
                output_2 = gr.Textbox(label="Proper Nouns")

        with gr.Row():

            clear_button = gr.Button("Clear")
            extract_button = gr.Button("Extract!")

        extract_button.click(extract_top_3,
                            inputs = input_article ,
                            outputs = [output_1, output_2])



demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://8e2c0bbf-c617-439d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://8e2c0bbf-c617-439d.gradio.live


## `Runall()`

In [94]:
def runall(dropdown_model, article_text):
    summary = summarizer(dropdown_model, article_text)
    keywords, proper_n = extract_top_3(article_text)

    return summary, keywords, proper_n


some_examples = [

        ["T5", """
    Carrefour livraison service honteux ! Ne livre jamais dans les temps, plusieurs heures de retard et si on a le malheur de travailler et de ne pas être chez nous toute la journée on peut dire au revoir à notre commande !
Service client délocalisé qui insulte et raccroche au nez quand on demande ou est notre commande !
Je ne recommande absolument pas, j'ai pourtant laisser plusieurs chance mais à chaque fois extrêmement déçu...
    """],
    ["T5", """
    Très mauvaise expérience !!!
Commande livré par terre sans rien dire alors que nous étions présents.
En plein centre ville de Dijon.
Merci pour l’hygiène et la compétence du livreur. 
Sans parler des surgelés en plein soleil qui ont finis à la poubelle.
Merci Carrefour Quetigny.
    """]
    
]

In [95]:
with gr.Blocks() as demo:
    gr.Markdown("Summarize the article text.")

    with gr.Tab("Summarization + Extraction"):
        with gr.Row():
            with gr.Column():
                input_models = gr.Dropdown(['T5'], value = 'T5')
                input_article = gr.TextArea(label = 'Article to be summarized')

            with gr.Column():
                summarized_output = gr.TextArea(label= 'Generated summa')
                output_keyword = gr.Textbox(label="Top 3 keywords")
                output_proper_nouns = gr.Textbox(label="Proper Nouns")

        with gr.Row():
            clear_button = gr.Button("Clear")
            summarize_button = gr.Button("Summarize!")
            extract_button = gr.Button("Extract!")
            run_button = gr.Button("Run all!")

        examples = gr.Examples(examples= some_examples,
                                     inputs=[input_models, input_article])


        summarize_button.click(summarizer,
                            inputs = [input_models, input_article] ,
                            outputs = summarized_output)
        

        extract_button.click(extract_top_3,
                            inputs = input_article ,
                            outputs = [output_keyword, output_proper_nouns])
        
        run_button.click(runall,
                         inputs = [input_models, input_article],
                         outputs = [summarized_output, output_keyword, output_proper_nouns ])
        
        clear_button.click(clear_input,
                        outputs = [input_models, input_article])

    with gr.Tab("Summarization"):
        with gr.Row():
            with gr.Column():
                input_models = gr.Dropdown(['camembert', 'T5'])
                input_article = gr.TextArea(label = 'Article to be summarized')
            with gr.Column():
                summarized_output = gr.TextArea(label= 'Generated summa')
        with gr.Row():

            clear_button = gr.Button("Clear")
            summarize_button = gr.Button("Summarize!")
            
        summarize_button.click(summarizer,
                            inputs = [input_models, input_article] ,
                            outputs = summarized_output)
        
        clear_button.click(clear_input,
                        outputs = [input_models, input_article])
    
        example = "Un nuage de fumée juste après l’explosion, le 1er juin 2019. Une déflagration dans une importante usine d’explosifs du centre de la Russie a fait au moins 79 blessés samedi 1er juin. L’explosion a eu lieu dans l’usine Kristall à Dzerzhinsk, une ville située à environ 400 kilomètres à l’est de Moscou, dans la région de Nijni-Novgorod. « Il y a eu une explosion technique dans l’un des ateliers, suivie d’un incendie qui s’est propagé sur une centaine de mètres carrés », a expliqué un porte-parole des services d’urgence. Des images circulant sur les réseaux sociaux montraient un énorme nuage de fumée après l’explosion. Cinq bâtiments de l’usine et près de 180 bâtiments résidentiels ont été endommagés par l’explosion, selon les autorités municipales. Une enquête pour de potentielles violations des normes de sécurité a été ouverte. Fragments de shrapnel Les blessés ont été soignés après avoir été atteints par des fragments issus de l’explosion, a précisé une porte-parole des autorités sanitaires citée par Interfax. « Nous parlons de blessures par shrapnel d’une gravité moyenne et modérée », a-t-elle précisé. Selon des représentants de Kristall, cinq personnes travaillaient dans la zone où s’est produite l’explosion. Elles ont pu être évacuées en sécurité. Les pompiers locaux ont rapporté n’avoir aucune information sur des personnes qui se trouveraient encore dans l’usine."
        examples = gr.Examples(examples=[ ["camembert",example],
                                        ["T5",example]],
                                        inputs=[input_models, input_article])
        
    with gr.Tab("Key word extraction"):
        

        with gr.Row():
            with gr.Column():
                input_article = gr.TextArea(label = 'Article to be extracted')
            with gr.Column():
                output_1 = gr.Textbox(label="Top 3 keywords")
                output_2 = gr.Textbox(label="Proper Nouns")

        with gr.Row():

            clear_button = gr.Button("Clear")
            extract_button = gr.Button("Extract!")

        extract_button.click(extract_top_3,
                            inputs = input_article ,
                            outputs = [output_1, output_2])



demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://2c9183ae-5cc6-4d17.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://2c9183ae-5cc6-4d17.gradio.live


Carrefour livraison service honteux ! Ne livre jamais dans les temps, plusieurs heures de retard et si on a le malheur de travailler et de ne pas être chez nous toute la journée on peut dire au revoir à notre commande !
Service client délocalisé qui insulte et raccroche au nez quand on demande ou est notre commande !
Je ne recommande absolument pas, j'ai pourtant laisser plusieurs chance mais à chaque fois extrêmement déçu...Très mauvaise expérience !!!
Commande livré par terre sans rien dire alors que nous étions présents.
En plein centre ville de Dijon.
Merci pour l’hygiène et la compétence du livreur. 
Sans parler des surgelés en plein soleil qui ont finis à la poubelle.
Merci Carrefour Quetigny.

---